# Battle of Neighbourhoods - Week 1







# Part 1 - A description of the problem and a discussion of the background

> ### 1.1 Description of the Problem

The population of Dubai has grown considerably over the last decades. Dubai is very diverse. It called Pearl of the Gulf. In Dubai, you can get variety of fresh food from supplies from Italy.
 
There are many fine restaurants in Dubai – Asian, Middle Eastern, Latin and American restaurants, it is hard to find a good place to dine in.


> ### 1.2 Discussion of the Background

My client is a successful restaurant chain in Napoli; he is looking to expand operation into Middle East starting in Emirates. His target is luxury restaurant
Since the Emirates demography has many Emirates, my client needs deeper insight from available data in other to decide where to establish first. This company spends a lot on research and provides customers with data insight into the ingredients used at restaurants.


# Part 2 - A description of the data and how it will be used to solve the problem

> ### 2.1 Description of Data


Data will be from public data, Wikipedia and Foursquare.

Emirates Area consists of 7 Emirates which has some specific popular areas < https://en.wikipedia.org/wiki/List_of_cities_in_the_United_Arab_Emirates >

#### A sample of the web scrapped of the Wikipedia page 

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [60]:
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_cities_in_Italy'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0'}
wikipedia_page = requests.get(wikipedia_link, headers = headers)
wikipedia_page

<Response [200]>

In [61]:
soup = BeautifulSoup(wikipedia_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody
rows = table.find_all('tr')
columns = [i.text.replace('\n', '')
           for i in rows[0].find_all('th')]
df = pd.DataFrame(columns = columns)

In [62]:
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df = df.append(pd.Series(values, index = columns), ignore_index = True)

        df

In [63]:
     df.head()

,Rank,City,2011 Census,2017 Estimate,Change,Region
0,1,Rome,"2,617,175","2,872,800",+9.77%,Lazio
1,2,Milan,"1,242,123","1,366,180",+9.99%,Lombardy
2,3,Naples,"962,003","966,144",+0.43%,Campania
3,4,Turin,"872,367","882,523",+1.16%,Piedmont
4,5,Palermo,"657,651","668,405",+1.64%,Sicily


In [50]:
df['City'] = df['City'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))

In [64]:
df0 = df.drop('City', axis=1).join(df['City'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('City'))

In [65]:
df1 = df0[['Rank', 'City', 'Region']].reset_index(drop=True)

In [67]:
df1.head()

,Rank,City,Region
0,1,Rome,Lazio
1,2,Milan,Lombardy
2,3,Naples,Campania
3,4,Turin,Piedmont
4,5,Palermo,Sicily


In [68]:
df2 = df1

In [71]:
df21 = df2[df2['Region'].str.contains('Lazio')]

In [72]:
df21.head(10)

,Rank,City,Region
0,1,Rome,Lazio
31,32,Latina,Lazio
57,58,Guidonia Montecelio,Lazio
69,70,Fiumicino,Lazio
75,76,Aprilia,Lazio
87,88,Viterbo,Lazio
94,95,Pomezia,Lazio
114,115,Tivoli,Lazio
120,121,Anzio,Lazio
129,130,Velletri,Lazio


In [73]:
df3 = df21[['Rank', 'City', 'Region']].reset_index(drop=True)

In [75]:
df_Lazio = df3

In [76]:
!pip -q install geocoder
import geocoder

In [77]:
'''Geocoder starts here'''
'''Defining a function to use --> get_latlng()'''
def get_latlng(arcgis_geocoder):
    
   
    lat_lng_coords = None
    
   
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Lazio, Italy'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords
'''Geocoder ends here'''

'Geocoder ends here'

In [81]:
City_codes = df['City']    
coordinates = [get_latlng(City_code) for City_code in City_codes.tolist()]

In [82]:
df_Lazio_loc = df_Lazio

df_Lazio_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_Lazio_loc['Latitude'] = df_Lazio_coordinates['Latitude']
df_Lazio_loc['Longitude'] = df_Lazio_coordinates['Longitude']

In [83]:
df_Lazio_loc.head()

,Rank,City,Region,Latitude,Longitude
0,1,Rome,Lazio,41.903220,12.495650
1,32,Latina,Lazio,41.491160,13.830130
2,58,Guidonia Montecelio,Lazio,40.776345,14.395212
3,70,Fiumicino,Lazio,45.001990,7.549558
4,76,Aprilia,Lazio,37.972820,13.186320


*The data output df_Lazio_loc will be used for further analysis in week 2.*

### 2.1.2 Dataset 2:

The Foursquare API will be used to obtain the geographical location data, to explore Lazio Region

### 2.2 How data will be used to solve the problem

The data from the datasets 1 and 2 exploring by considering the venues within the neighbourhood of Lazio Region